In [1]:
import cv2 as cv
import numpy as np
import math
from itertools import combinations
from scipy import ndimage as ndi

In [2]:
img = cv.imread(cv.samples.findFile('Goalball.jpg'))
height, width, _ = img.shape
blank_img = np.zeros((height,width,3), np.uint8)
blank_img = cv.cvtColor(blank_img,cv.COLOR_BGR2GRAY)

In [3]:
hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

#create a mask for the camp colour using inRange function
mask = cv.inRange(hsv, (0,0,100), (50,150,200))

#perform bitwise and on the original image arrays using the mask
img_masked = cv.bitwise_and(img, img, mask=mask)

In [4]:
Z = img_masked.reshape((-1,3))
# convert to np.float32
Z = np.float32(Z)
# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 2
ret,label,center=cv.kmeans(Z,K,None,criteria,10,cv.KMEANS_RANDOM_CENTERS)
# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))
res2 = cv.dilate(res2, None)
res2 = cv.cvtColor(res2,cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(res2, 10, 255, cv.THRESH_BINARY)
edges = cv.Canny(thresh,150,300,apertureSize = 3, L2gradient=True)
edges = cv.dilate(edges, None)
cv.imwrite('Canny.jpg',edges)
cv.imwrite('res2.jpg',thresh)

True

In [5]:
highest_v = [0,None]
lowest_v = [3000, None]
lines = cv.HoughLinesP(edges,1,np.pi/180,50,minLineLength=100,maxLineGap=6)
for line in lines:
    x1,y1,x2,y2 = line[0]
    if x1 == x2:
        continue
    elif y1 == y2:
        continue
    else:
        m = (y2-y1)/(x2-x1)

        if abs(y2-y1) > 50 and abs(m) <2:
            b = y1-(m*x1)
            line_y = lambda x: math.floor((m*x)+b)
            line_x = lambda y: math.floor((y-b)/m)

            if line_y(height/2) > highest_v[0]:
                highest_v[0] = line_y(height/2)
                highest_v[1] = ((0,line_y(0)),(width,line_y(width)))
            if line_y(height/2) < lowest_v[0]:
                lowest_v[0] = line_y(height/2)
                lowest_v[1] = ((0,line_y(0)),(width,line_y(width)))

cv.line(blank_img,(highest_v[1][0]),(highest_v[1][1]),(255,255,255),1)
cv.line(blank_img,(lowest_v[1][0]),(lowest_v[1][1]),(255,255,255),1)
cv.line(img,(highest_v[1][0]),(highest_v[1][1]),(0,0,255),2, lineType=cv.LINE_AA)
cv.line(img,(lowest_v[1][0]),(lowest_v[1][1]),(0,0,255),2,lineType=cv.LINE_AA)

TypeError: ignored

In [ ]:
highest_h = 0
lowest_h = 30000
contours, _ = cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv.contourArea, reverse=True)
for i in range(3):
    quadra = contours[i]
    x,y,w,h = cv.boundingRect(quadra)
    if (y+h) > highest:
        highest = (y+h)
    if y > highest:
        highest = y
    if (y+h) < lowest:
        lowest = (y+h)
    if y < lowest:
        lowest = y
    # cv.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
cv.line(blank_img,(0,lowest),(width, lowest),(255,255,255),1)
cv.line(blank_img,(0,highest),(width, highest),(255,255,255),1)
cv.line(img,(0,lowest),(width, lowest),(0,0,255),2)
cv.line(img,(0,highest),(width, highest),(0,0,255),2)

array([[[ 45,  44,  71],
        [ 44,  44,  70],
        [ 43,  43,  69],
        ...,
        [ 31,  42,  77],
        [ 31,  42,  77],
        [ 31,  42,  77]],

       [[ 45,  44,  71],
        [ 45,  42,  68],
        [ 42,  42,  69],
        ...,
        [ 31,  42,  77],
        [ 31,  42,  77],
        [ 31,  42,  77]],

       [[ 45,  45,  70],
        [ 42,  43,  67],
        [ 40,  40,  65],
        ...,
        [ 31,  42,  77],
        [ 31,  42,  77],
        [ 31,  42,  77]],

       ...,

       [[ 86, 126,  46],
        [ 86, 126,  46],
        [ 86, 126,  46],
        ...,
        [ 80, 116,  43],
        [ 80, 116,  43],
        [ 80, 116,  43]],

       [[ 86, 126,  46],
        [ 86, 126,  46],
        [ 86, 126,  46],
        ...,
        [ 80, 116,  43],
        [ 80, 116,  43],
        [ 80, 116,  43]],

       [[ 86, 126,  46],
        [ 86, 126,  46],
        [ 86, 126,  46],
        ...,
        [ 80, 116,  43],
        [ 80, 116,  43],
        [ 80, 116,  43]]

In [ ]:
gray = np.float32(blank_img)
dst = cv.cornerHarris(gray,2,3,0.01)
blank_img[:] = 0
blank_img[dst==dst.max()] = 255
corners = np.argwhere(blank_img)
corners = corners.astype(np.int32)
corners = corners.reshape((-1,1,2))
print(corners) # Coordenadas dos Cantos da Quadra

[[[ 308  455]]

 [[ 308 1431]]

 [[ 886 1737]]

 [[ 889  142]]]


In [ ]:
cv.imwrite('Hough3.jpg',img)
cv.imwrite('Hough_Blank.jpg',blank_img)

True